In [1]:
from configparser import ConfigParser
from census import Census
import pandas as pd
import numpy as np
import time
import os

# Census Data Pull

This analysis utilizes data from the U.S. Census Bureau's American Community Survey (ACS), an annual survey covering social, economic, demographic, and housing characteristics. The Census Bureau maintains an API for easy access to their public datasets, including the ACS.

Request a Census Data API Key [here](https://api.census.gov/data/key_signup.html). For more information on the census library, a wrapper for the Census API, visit [Python Package Index](https://pypi.org/project/census/).

In [2]:
# retrieve API from configuration file
config = ConfigParser()
config.read('../config.ini')
API_KEY = config['CENSUS']['API']
c = Census(API_KEY)

See the list of variables available from the [Census Bureau](https://api.census.gov/data/2022/acs/acs5/variables.html).

In [3]:
variables = (
    'GEO_ID',
    'TRACT',
    'COUNTY',
    'STATE',
    'NAME',
    "B01003_001E", # Total Population
    # Economic
    "B17001_002E", # Poverty
    "B19013_001E", # Median Household Income
    "B23025_002E", # Total in Labor Force
    "B23025_004E", # Employed - Civilian Labor Force
    "B23025_006E", # Employed - Armed Forces
    # Race & Ethnicity
    "B02001_002E", # White
    "B02001_003E", # Black or African American 
    "B02001_004E", # American Indian and Alaska Native
    "B02001_005E", # Asian
    "B02001_006E", # Native Hawaiian and other Pacific Islander 
    "B02001_007E", # Some other race
    "B02001_008E", # Two or more races
    "B03001_003E", # Hispanic or Latino 
    # Age & Sex
    "B01001_002E", # Male
    "B01001_026E", # Female
    "B01001_003E", # Male < 5
    "B01001_004E", # Male 5-9
    "B01001_005E", # Male 10-14
    "B01001_006E", # Male 15-17
    "B01001_007E", # Male 18-19
    "B01001_008E", # Male 20
    "B01001_009E", # Male 21
    "B01001_010E", # Male 22-24
    "B01001_011E", # Male 25-29
    "B01001_012E", # Male 30-34
    "B01001_013E", # Male 35-39
    "B01001_014E", # Male 40-44
    "B01001_015E", # Male 45-49
    "B01001_016E", # Male 50-54
    "B01001_017E", # Male 55-59
    "B01001_018E", # Male 60-61
    "B01001_019E", # Male 62-64
    "B01001_020E", # Male 65-66
    "B01001_021E", # Male 67-69
    "B01001_022E", # Male 70-74
    "B01001_023E", # Male 75-79
    "B01001_024E", # Male 80-84
    "B01001_025E", # Male 85+
    "B01001_027E", # Female < 5
    "B01001_028E", # Female 5-9
    "B01001_029E", # Female 10-14
    "B01001_030E", # Female 15-17
    "B01001_031E", # Female 18-19
    "B01001_032E", # Female 20
    "B01001_033E", # Female 21
    "B01001_034E", # Female 22-24
    "B01001_035E", # Female 25-29
    "B01001_036E", # Female 30-34
    "B01001_037E", # Female 35-39
    "B01001_038E", # Female 40-44
    "B01001_039E", # Female 45-49
    "B01001_040E", # Female 50-54
    "B01001_041E", # Female 55-59
    "B01001_042E", # Female 60-61
    "B01001_043E", # Female 62-64
    "B01001_044E", # Female 65-66
    "B01001_045E", # Female 67-69
    "B01001_046E", # Female 70-74
    "B01001_047E", # Female 75-79
    "B01001_048E", # Female 80-84
    "B01001_049E", # Female 85+
    # Disability status
    "B18101_004E", # Male, <5 with a Disability
    "B18101_007E", # Male, 5-17 with a Disability
    "B18101_010E", # Male, 18-34 with a Disability
    "B18101_013E", # Male, 35-64 with a Disability
    "B18101_016E", # Male, 65-75 with a Disability
    "B18101_019E", # Male, 75+ with a Disability
    "B18101_023E", # Female, <5 with a Disability
    "B18101_026E", # Female, 5-17 with a Disability
    "B18101_029E", # Female, 18-34 with a Disability
    "B18101_032E", # Female, 35-64 with a Disability
    "B18101_035E", # Female, 65-75 with a Disability
    "B18101_038E", # Female, 75+ with a Disability
    # Housing
    "B25001_001E", # Total Housing Units
    "B25002_003E", # Vacant Housing Units
    "B25003_002E"  # Owner-occupied housing units
    
)

We will be pulling data for all census tracts in the U.S. The Census API requires that census tract queries be pulled one state at a time, so we first need a list of the state's [FIPS](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) codes.

In [4]:
# FIPS codes for the 50 states and Washington D.C.
state_fips = [str(n).zfill(2) for n in range(1,57) if n not in (3,7,14,43,52)]


Make API call for each state in state_fips. Append results to us_df dataframe.

In [5]:
us_df = pd.DataFrame()

# make Census API call
print('Sending request to Census API ...')

for state in state_fips:
    response = c.acs5.state_county_tract(
                        fields = variables,
                        state_fips = state,
                        county_fips = Census.ALL,
                        tract = Census.ALL)

    # convert results to pd dataframe
    state_df = pd.DataFrame(response)

    # concat to final df
    us_df = pd.concat([us_df, state_df], ignore_index = True)

    time.sleep(1)

print('Request sent.')

Sending request to Census API ...
Request sent.


In [6]:
us_df.head(3)

,GEO_ID,TRACT,COUNTY,STATE,NAME,B01003_001E,B17001_002E,B19013_001E,B23025_002E,B23025_004E,...,B18101_019E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B25001_001E,B25002_003E,B25003_002E
0,1400000US01001020100,020100,001,01,Census Tract 201; Autauga County; Alabama,1865.0,286.0,60563.0,738.0,713.0,...,68.0,0.0,6.0,9.0,75.0,18.0,38.0,733.0,33.0,519.0
1,1400000US01001020200,020200,001,01,Census Tract 202; Autauga County; Alabama,1861.0,105.0,57460.0,947.0,868.0,...,0.0,0.0,28.0,13.0,120.0,67.0,6.0,680.0,136.0,429.0
2,1400000US01001020300,020300,001,01,Census Tract 203; Autauga County; Alabama,3492.0,352.0,77371.0,1808.0,1748.0,...,24.0,0.0,0.0,5.0,126.0,64.0,77.0,1431.0,126.0,912.0


## Clean Data

In [7]:
# create final dataframe
census = us_df.loc[:,['GEO_ID','NAME']].copy(deep = True)

# clean data
census['GEO_ID'] =      census['GEO_ID'].str[-11:]   
census['Pop'] =         us_df['B01003_001E'].astype(int)
census['Hous_Units'] =  us_df['B25001_001E'].astype(int)
census['Income'] =      np.where(us_df['B19013_001E'] == -666666666.0, None, us_df['B19013_001E'])
census['Poverty%'] =    us_df['B17001_002E']/census['Pop']
census['White%'] =      us_df['B02001_002E']/census['Pop'] 
census['Black%'] =      us_df['B02001_003E']/census['Pop'] 
census['Hispanic%'] =   us_df['B03001_003E']/census['Pop']
census['Asian%'] =      us_df['B02001_004E']/census['Pop']
census['TwoOrMore%'] =  us_df['B02001_008E']/census['Pop']  
census['Male%'] =       us_df['B01001_002E']/census['Pop']
census['Female%'] =     us_df['B01001_026E']/census['Pop']
census['Under18%'] =    us_df[['B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_027E',
                              'B01001_028E','B01001_029E','B01001_030E']].sum(axis=1)/census['Pop']   
census['65+%'] =        us_df[['B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E',
                              'B01001_025E','B01001_044E','B01001_045E','B01001_046E','B01001_047E',
                              'B01001_048E','B01001_049E']].sum(axis=1)/census['Pop']
census['Employment%'] = (us_df['B23025_004E']+us_df['B23025_006E'])/us_df['B23025_002E']
census['Disability%'] =  us_df[['B18101_004E','B18101_007E','B18101_010E','B18101_013E','B18101_016E',
                               'B18101_019E','B18101_023E','B18101_026E','B18101_029E','B18101_032E',
                               'B18101_035E','B18101_038E']].sum(axis=1)/census['Pop']
census['VacantHous%'] =  us_df['B25002_003E']/census['Hous_Units']
census['OwnOcpHous%'] =  us_df['B25003_002E']/census['Hous_Units']


In [8]:
census.head(3)

,GEO_ID,NAME,Pop,Hous_Units,Income,Poverty%,White%,Black%,Hispanic%,Asian%,TwoOrMore%,Male%,Female%,Under18%,65+%,Employment%,Disability%,VacantHous%,OwnOcpHous%
0,01001020100,Census Tract 201; Autauga County; Alabama,1865,733,60563.0,0.153351,0.804290,0.111528,0.043432,0.0,0.055228,0.519035,0.480965,0.239678,0.194638,0.974255,0.200000,0.04502,0.708049
1,01001020200,Census Tract 202; Autauga County; Alabama,1861,680,57460.0,0.056421,0.363246,0.559914,0.001075,0.0,0.072542,0.544331,0.455669,0.098872,0.157442,0.946146,0.180548,0.20000,0.630882
2,01001020300,Census Tract 203; Autauga County; Alabama,3492,1431,77371.0,0.100802,0.691008,0.250859,0.012600,0.0,0.048110,0.460195,0.539805,0.261741,0.134593,0.981748,0.158362,0.08805,0.637317


## Export Results

In [9]:
census.to_csv("../data/Census_ACS5_Export.csv", index = False)